In [33]:
from datetime import datetime
import nasdaqdatalink
import quandl
import yfinance as yf
import pandas as pd
from fredapi import Fred


In [34]:
import json


def GenerateHighchartVar(df, fieldX, fieldY):
    df.reset_index()
    init = '[' + '\n'
    s = ''
    
    for i in range(len(df)):
        s = s + '[' + str(df.loc[i, fieldX].timestamp() * 1000) + ',' + str(df.loc[i, fieldY]) + '],' + '\n'
    
    s = s[:-2]
    s = init + s + "]"
    
    return s

def generateJSONDataFile(fileName, content):
    with open("data/data_" + fileName+ ".json", "w") as outfile:
        outfile.write(content)
        
def generateMetadataFile(dict, fileName):
    with open('data/meta_' + fileName + '.js', 'w') as convert_file:
     convert_file.write('var meta = ')
     convert_file.write(json.dumps(dict))
     convert_file.write(';')



In [35]:
# Yahoo Data
# https://www.ssga.com/library-content/products/fund-docs/etfs/us/information-schedules/spdr-etf-listing.pdf
tickers_WorldIndex = {'^GSPC' : 'SP500','^DJI' : 'Dow Jones','^IXIC' : 'Nasdaq','^RUT' : 'Russell 2000','^VIX' : 'VIX','^FTSE' : 'FTSE 100','^N225' : 'Nikkei 225','^HSI' : 'Hang Seng Index'}
tickers_ccy = {'DX-Y.NYB' : 'USD', 'EURUSD=X' : 'EURUSD','JPY=X' : 'USDJPY','GBPUSD=X' : 'GBPUSD', 'AUDUSD=X' : 'AUDUSD', 'NZDUSD=X' : 'NZDUSD','CNY=X' : 'CNY','CAD=X' : 'USDCAD'}
tickers_commodities = {'GC=F' : 'Gold','SI=F' : 'Silver','CL=F' : 'Crude'}
tickers_treasury = {'ZT=F' : 'US 2-Year Note','ZN=F' : 'US 10-Year Note','ZB=F' : 'US Treasury'}

tickers_sector = {'XLC' : 'Communication Service','XLP' : 'Consumer Staples','XLY' : 'Consumer Discretionary','XLE' : 'Energy','XLF' : 'Financial','XLV' : 'Health Care','XLI' : 'Industrial','XLB' : 'Materials','XLRE' : 'Real Estate','XLK' : 'Technology','XLU' : 'Utilities'}
tickers_style = {'SPTM' : 'SP 1500','SPLG' : 'Large Cap','SPMD' : 'Mid Cap','SPSM'  : 'Small Cap','SPYG'  : 'Growth','SPYV' : 'Value','SPYD' : 'High Dividend Yield'}

tickers_yahoo = {**tickers_WorldIndex, **tickers_ccy, **tickers_commodities, **tickers_treasury, **tickers_sector, **tickers_style}


asOfDateTime = datetime.now()
asOfDateTimeStr = asOfDateTime.strftime("%d/%m/%Y %H:%M:%S")

for t in tickers_yahoo:
    name = t.replace('^','').replace('=F','').replace('=X','').replace('DX-Y.NYB','DXY')
    rawData = yf.download(t)
    #indexedData = rawData['Adj Close'].tail(950).reset_index()
    indexedData = rawData['Adj Close'].reset_index()
    indexedData.columns = ['Date','Value']
    highChartTS = GenerateHighchartVar(indexedData, 'Date','Value')
    generateJSONDataFile(name, highChartTS)
    
    meta = {'name': name,
            'displayName': tickers_yahoo[t],
        'currentUpdate': (indexedData.tail(1)['Date'].item()).strftime('%d-%m-%y'),
        'currentValue' : indexedData.tail(1)['Value'].item(),
        'minDate' : (indexedData.min()['Date']).strftime('%d-%m-%y'),
        'minValue' : indexedData.min()['Value'],
        'maxDate' : (indexedData.max()['Date']).strftime('%d-%m-%y'),
        'maxValue' : indexedData.max()['Value'],
        'lastUpdate' : asOfDateTimeStr,
        'dataFilename' : '/macroview/data/data_' + name + '.json'
        }
    
    generateMetadataFile(meta, name)
    
print('Successfully download Yahoo data')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [24]:
# Nasdaq Data
tickers_spRatio = {"MULTPL/SHILLER_PE_RATIO_MONTH" : 'Shiller PE Ratio',"MULTPL/SP500_DIV_YIELD_MONTH" : 'S&P500 Dividend Yield',"MULTPL/SP500_PE_RATIO_MONTH" : 'S&P 500 PE Ratio',"MULTPL/SP500_EARNINGS_YIELD_MONTH" : 'S&P 500 Earning Yield',"MULTPL/SP500_PBV_RATIO_QUARTER" : 'S&P 500 Price to Book Ratio',"MULTPL/SP500_PSR_QUARTER" : 'S&P 500 Price to Sales Ratio'}
tickers_worldInflationYoY = {"RATEINF/INFLATION_USA" : 'US Inflation',"RATEINF/INFLATION_GBR" : 'UK Inflation',"RATEINF/INFLATION_EUR" : 'Euro Area Inflation',"RATEINF/INFLATION_JPN" : 'Japan Inflation'}

tickers_allNasdaq = {**tickers_spRatio, **tickers_worldInflationYoY}

asOfDateTime = datetime.now()
asOfDateTimeStr = asOfDateTime.strftime("%d/%m/%Y %H:%M:%S")

for t in tickers_allNasdaq:
    rawData = quandl.get(t, authtoken="cza_RyNfSzs9o1Z2QBs4")
    #indexedData = rawData.tail(950).reset_index()
    indexedData = rawData.reset_index()
    highChartTS = GenerateHighchartVar(indexedData, 'Date','Value')
    generateJSONDataFile(t.replace('/','_'), highChartTS)
    
    meta = {'name': t.replace('/','_'),
            'displayName': tickers_allNasdaq[t],
        'currentUpdate': (indexedData.tail(1)['Date'].item()).strftime('%d-%m-%Y'),
        'currentValue' : indexedData.tail(1)['Value'].item(),
        'minDate' : (indexedData.min()['Date']).strftime('%d-%m-%Y'),
        'minValue' : indexedData.min()['Value'],
        'maxDate' : (indexedData.max()['Date']).strftime('%d-%m-%Y'),
        'maxValue' : indexedData.max()['Value'],
        'lastUpdate' : asOfDateTimeStr,
        'dataFilename' : '/macroview/data/data_' + t.replace('/','_') + '.json'
        }
    
    generateMetadataFile(meta, t.replace('/','_'))


print('Successfully download Nasdaq data')



Successfully download Nasdaq data


In [26]:
# Fred Data

fred = Fred(api_key='d79cebb1e12819cd44ed96cc291f0f72')

def generateFredMeta(indexedData, name = '', displayName = ''):
    asOfDateTime = datetime.now()
    asOfDateTimeStr = asOfDateTime.strftime("%d/%m/%Y %H:%M:%S")
    meta = {'name': name,
            'displayName': displayName,
            'currentUpdate': (indexedData.tail(1)['Date'].item()).strftime('%d-%m-%y'),
            'currentValue' : indexedData.tail(1)['Value'].item(),
            'minDate' : (indexedData.min()['Date']).strftime('%d-%m-%y'),
            'minValue' : indexedData.min()['Value'],
            'maxDate' : (indexedData.max()['Date']).strftime('%d-%m-%y'),
            'maxValue' : indexedData.max()['Value'],
            'lastUpdate' : asOfDateTimeStr,
            'dataFilename' : '/macroview/data/data_' + name + '.json'
            }
    return meta

In [27]:
coreCPI = fred.get_series('CPILFESL')
coreCPIYOY = coreCPI.pct_change(periods=12)*100
coreCPIYOY = coreCPIYOY.dropna()
coreCPIYOY_reset = coreCPIYOY.reset_index()
coreCPIYOY_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(coreCPIYOY_reset, 'Date','Value')
meta = generateFredMeta(coreCPIYOY_reset, 'coreCPIYOY', 'US Core CPI')
generateMetadataFile(meta, 'coreCPIYOY')
generateJSONDataFile('coreCPIYOY', highChartTS)

In [28]:
headlineCPI = fred.get_series('CPIAUCSL')
headlineCPIYOY = headlineCPI.pct_change(periods=12)*100
headlineCPIYOY = headlineCPIYOY.dropna()
headlineCPIYOY_reset = headlineCPIYOY.reset_index()
headlineCPIYOY_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(headlineCPIYOY_reset, 'Date','Value')
meta = generateFredMeta(headlineCPIYOY_reset, 'headlineCPIYOY', 'US Headline CPI')
generateMetadataFile(meta, 'headlineCPIYOY')
generateJSONDataFile('headlineCPIYOY', highChartTS)

In [29]:
#10y2y yield spread
series = 'T10Y2Y' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US 10-2 Year Treasury Yield Spread')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)


In [30]:
#Real Yield
series = 'DFII10' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Real Yield')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)



In [31]:
#Nominal Yield
series = 'DGS10' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Nominal Yield')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)


In [32]:
#Breakeven Rate
series = 'T10YIE' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US 10-Year Breakeven Inflation Rate')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)

print('Successfully download Fred data')


Successfully download Fred data


## Testing

In [92]:
# https://colab.research.google.com/drive/1S7bAG8HAJ6B-KkWlClmeOAc9V9HJOPZB?usp=sharing#scrollTo=ti8GH8YikINi

import yfinance as yf
import pandas as pd


In [ ]:
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_table = payload[0]
sp500_table

KeyboardInterrupt: 

In [ ]:
sp500_tickers = sp500_table.Symbol.str.upper().values
sp500_names = sp500_table.Security.values
sp500_sectors = sp500_table["GICS Sector"].values
sp500_sub_sectors = sp500_table["GICS Sub-Industry"].values

sp500_names_mapping = dict(zip(sp500_tickers, sp500_names))
sp500_sector_mapping = dict(zip(sp500_names, sp500_sectors))
sp500_sub_sector_mapping = dict(zip(sp500_names, sp500_sub_sectors))


In [ ]:
tickers = list(sp500_tickers)
prices = yf.download(tickers, start="2020-01-01", end="2021-12-31", interval='1d')
prices = prices["Adj Close"]
prices = prices.rename(columns=sp500_names_mapping)
prices

[*********************100%***********************]  505 of 505 completed

3 Failed downloads:
- CEG: Data doesn't exist for startDate = 1577808000, endDate = 1640880000
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


,Agilent Technologies,American Airlines Group,Advance Auto Parts,Apple,AbbVie,AmerisourceBergen,Abiomed,Abbott,Accenture,Adobe,...,Wynn Resorts,Xcel Energy,ExxonMobil,Dentsply Sirona,Xylem,Yum! Brands,Zimmer Biomet,Zebra,Zions Bancorp,Zoetis
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,84.168228,28.574404,155.080658,72.245934,79.233513,82.006271,170.589996,83.914215,204.183365,329.809998,...,137.714386,59.756420,60.208515,55.656384,76.772598,96.749886,143.179916,255.440002,48.534744,130.665115
2020-01-02,84.799675,28.982893,154.354446,73.894325,80.137344,82.064148,168.809998,84.001175,203.776093,334.429993,...,142.405029,58.711700,61.174896,55.990772,77.805458,98.132988,142.720795,259.140015,49.020840,132.432312
2020-01-03,83.438148,27.548195,154.364105,73.175926,79.376694,81.032066,166.820007,82.977112,203.436752,331.809998,...,140.292755,58.994057,60.683079,55.371174,78.263420,97.825638,142.347702,256.049988,48.329098,132.452057
2020-01-06,83.684799,27.219410,151.827209,73.759010,80.003113,82.218460,179.039993,83.411858,202.108246,333.709991,...,140.015091,58.909351,61.149021,55.676060,77.756729,97.768013,141.525040,258.010010,47.740170,131.435196
2020-01-07,83.941315,27.119778,150.026199,73.412117,79.546722,81.630089,180.350006,82.948143,197.744751,333.390015,...,140.679504,58.786999,60.648567,55.971111,77.464401,97.940918,141.400711,256.470001,47.450382,131.879440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,157.592438,18.260000,230.452118,176.055695,131.723129,129.109039,352.209991,138.670883,402.267090,569.619995,...,88.879997,66.148552,60.350952,55.322369,117.112770,134.714600,123.051254,582.409973,62.041107,242.121170
2021-12-27,158.531204,18.170000,234.790527,180.100540,133.029587,131.471329,357.829987,140.962799,414.255981,577.679993,...,87.580002,66.357094,61.211414,55.841362,117.900162,137.372253,124.087379,606.330017,62.687782,246.114746
2021-12-28,158.970612,18.540001,236.408752,179.061859,133.009781,131.929840,357.440002,139.976273,414.196136,569.359985,...,86.459999,67.151550,61.013607,55.921204,119.126106,137.342392,124.475731,597.320007,62.787273,243.858383


In [ ]:
prices.to_csv('sp500prices.csv')

In [ ]:
# https://www.ssga.com/library-content/products/fund-docs/etfs/us/information-schedules/spdr-etf-listing.pdf
tickers_WorldIndex = ['^GSPC','^DJI','^IXIC','^RUT','^VIX','^FTSE','^N225','^HSI']
tickers_ccy = ['EURUSD=X','JPY=X','GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X','CNY=X','CAD=X']
tickers_commodities = ['GC=F','SI=F','CL=F']
tickers_treasury = ['ZT=F','ZN=F','ZB=F']

tickers_sector = ['XLC','XLP','XLY','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
tickers_style = ['SPTM','SPLG','SPMD','SPSM','SPYG','SPYV','SPYD']

prices = yf.download(tickers_WorldIndex+tickers_ccy+tickers_commodities+tickers_treasury+tickers_sector+tickers_style)
prices['Adj Close'].reset_index()


[*********************100%***********************]  31 of 31 completed

1 Failed download:
- XLKXLU: No data found, symbol may be delisted


,Date,AUDUSD=X,CAD=X,CL=F,CNY=X,EURUSD=X,GBPUSD=X,GC=F,JPY=X,NZDUSD=X,...,ZN=F,ZT=F,^DJI,^FTSE,^GSPC,^HSI,^IXIC,^N225,^RUT,^VIX
0,1950-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.660000,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.850000,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.930000,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.980000,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,17.080000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18664,2022-03-22,0.739579,1.25880,111.760002,6.3545,1.101722,1.316829,1920.699951,119.594002,0.687971,...,122.921875,106.687500,34807.460938,7476.700195,4511.609863,21889.279297,14108.820312,27224.109375,2088.340088,22.940001
18665,2022-03-23,0.746230,1.25828,114.930000,6.3653,1.102974,1.326647,1936.599976,121.339996,0.696432,...,123.140625,106.761719,34358.500000,7460.600098,4456.240234,22154.080078,13922.599609,28040.160156,2052.209961,23.570000
18666,2022-03-24,0.749918,1.25592,112.339996,6.3713,1.100631,1.320603,1961.599976,121.126999,0.697632,...,122.921875,106.714844,34707.941406,7467.399902,4520.160156,21945.949219,14191.839844,28110.390625,2075.439941,21.670000
18667,2022-03-25,0.751428,1.25204,113.900002,6.3663,1.100958,1.319575,1953.800049,122.346001,0.696549,...,121.546875,106.468750,34861.238281,7483.399902,4543.060059,21404.880859,14169.299805,28149.839844,2077.979980,20.809999


In [ ]:
prices['Adj Close'].to_csv('yahooData.csv')

In [ ]:
import investpy
data = investpy.economic_calendar(from_date='01/03/2022', to_date='01/09/2022')
data

,id,date,time,zone,currency,importance,event,actual,forecast,previous
0,173,01/03/2022,All Day,south korea,None,None,South Korea - Independence Day,None,None,None
1,202,01/03/2022,All Day,argentina,None,None,Argentina - Carnival,None,None,None
2,202,01/03/2022,All Day,brazil,None,None,Brazil - Carnival,None,None,None
3,202,01/03/2022,All Day,venezuela,None,None,Venezuela - Carnival,None,None,None
4,202,01/03/2022,All Day,ecuador,None,None,Ecuador - Carnival,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2911,216,31/08/2022,All Day,malaysia,None,None,Malaysia - National Day,None,None,None
2912,247,31/08/2022,All Day,india,None,None,India - Ganesh Chaturthi,None,None,None
2913,178,01/09/2022,All Day,slovakia,None,None,Slovakia - Constitution Day,None,None,None
2914,215,01/09/2022,All Day,vietnam,None,None,Vietnam - Public Holiday,None,None,None


In [ ]:
import dbnomics
worldGDP = dbnomics.fetch_series('WB', 'WDI', 'NY.GDP.MKTP.KD.ZG-1W')
worldGDP

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,indicator,country,indicator (label),country (label)
0,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,1960,1960-01-01,NA,NaN,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
1,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,1961,1961-01-01,3.80083,3.800832,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
2,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,1962,1962-01-01,5.32028,5.320280,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
3,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,1963,1963-01-01,5.19242,5.192421,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
4,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,1964,1964-01-01,6.56875,6.568749,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,2016,2016-01-01,2.82457,2.824569,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
57,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,2017,2017-01-01,3.39464,3.394638,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
58,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,2018,2018-01-01,3.2627,3.262697,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World
59,annual,WB,WDI,World Development Indicators,NY.GDP.MKTP.KD.ZG-1W,GDP growth (annual %) – World,2019,2019-01-01,2.60215,2.602147,NY.GDP.MKTP.KD.ZG,1W,GDP growth (annual %),World


In [ ]:
worldGDP[['period','value']]

,period,value
0,1960-01-01,NaN
1,1961-01-01,3.800832
2,1962-01-01,5.320280
3,1963-01-01,5.192421
4,1964-01-01,6.568749
...,...,...
56,2016-01-01,2.824569
57,2017-01-01,3.394638
58,2018-01-01,3.262697
59,2019-01-01,2.602147


In [ ]:
f = worldGDP[['period','value']]
dataFileName = 'worldGDP'
x = generateJSVar(f_reset, 'x1', 'period', isX=True)
y = generateJSVar(f_reset, 'data1','value', isX=False)
generateJSDataFile('data_'+ dataFileName,[x,y])